In [4]:
%matplotlib inline
import os
import pandas as pd
from glob import glob
import numpy as np

In [5]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
import pathlib

Using TensorFlow backend.


In [6]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
genres = 'classical jazz metal pop rock'.split()

In [ ]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
for g in genres:
    pathlib.Path(f'dataset/img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'dataset/img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

# Create a spectrogram

In [8]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = 'working/train/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [9]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('working/test/' + name + '.jpg')
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

# Extracting features

In [1]:
import csv

In [2]:
ids = []
labels = []

In [10]:
file = open('./dataset/data.csv', 'w', newline='')
header = ["id","class"]
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        create_spectrogram(songname,filename)
        
        # extracts the  genre name
        labels.append(filename.split('.')[0])
        ids.append(filename + '.jpg')
        


In [11]:
df = pd.DataFrame({'ID': ids, 'Class': labels})

In [12]:
df.head()

,ID,Class
0,classical.00046.au.jpg,classical
1,classical.00094.au.jpg,classical
2,classical.00030.au.jpg,classical
3,classical.00005.au.jpg,classical
4,classical.00010.au.jpg,classical


# Creating a data generator

In [32]:
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.backend import placeholder
from keras.layers import Conv2D, MaxPooling2D, InputLayer
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=.25,
)

In [33]:
train_generator = datagen.flow_from_dataframe(
    dataframe = df,
    directory="working/train/",
    x_col = "ID",
    y_col = "Class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(128, 128)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe = df,
    directory="working/train/",
    x_col = "ID",
    y_col = "Class",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(128, 128)
)

Found 375 images belonging to 5 classes.
Found 125 images belonging to 5 classes.


In [36]:
from keras import applications

def get_model(weights=False):
    vgg_model = applications.VGG16(weights='imagenet',
                                   include_top=False,
                                   input_shape=(128, 128, 3))

    # Creating dictionary that maps layer names to the layers
    layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

    # Getting output tensor of the last VGG layer that we want to include
    x = vgg_model.output

    # Stacking a new simple convolutional network on top of it    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='softmax')(x)

    # Creating new model. Please note that this is NOT a Sequential() model.
    from keras.models import Model
    custom_model = Model(input=vgg_model.input, output=x)
    
    if weights:
        custom_model.load(weights)
    
    return custom_model


# Do not forget to compile it
custom_model = get_model()
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy'])
custom_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

/home/jeremy/miniconda3/envs/cs4803-project/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [ ]:
#Fitting keras model, no test gen for now
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
custom_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=75
)

Epoch 1/75
 7/11 [==================>...........] - ETA: 15s - loss: 11.0170 - acc: 0.2277

In [ ]:
custom_model.save('vgg_sound_classifier.h5')

# Transfer Learning Boilerplate

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
SAMPLE_SONG_PATH = "working/train/classical.00000.au.jpg"
STYLE_SONG_PATH = "working/train/jazz.00000.au.jpg"

In [ ]:
TARGET_HEIGHT = 128
TARGET_WIDTH = 128
TARGET_SIZE = (TARGET_HEIGHT, TARGET_WIDTH)

In [ ]:
cImage = load_img(path=SAMPLE_SONG_PATH, target_size=TARGET_SIZE)
cImArr = img_to_array(cImage)
cImArr = np.expand_dims(cImArr, axis=0)
cImArr = K.variable(cImArr, dtype='float32')

In [ ]:
cImArr.shape

In [ ]:

sImage = load_img(path=SAMPLE_SONG_PATH, target_size=TARGET_SIZE)
sImArr = img_to_array(sImage)
sImArr = np.expand_dims(sImArr, axis=0)
sImArr = K.variable(sImArr, dtype='float32')

In [ ]:
generatedImage = np.random.randint(
    256, size=(1, TARGET_WIDTH, TARGET_HEIGHT, 3)).astype('float64')
generatedImagePlaceholder = K.placeholder(shape=(1, TARGET_WIDTH, TARGET_HEIGHT, 3))

In [ ]:
def getLayerIndexByName(model, layername):
    for idx, layer in enumerate(model.layers):
        if layer.name == layername:
            return idx
        
def get_feature_reps(x, layer_names, model):
    """
    Get feature representations of input x for one or more layers in a given model.
    """
    featMatrices = []
    inp = model.input
    
    outputs = [layer.output for layer in model.layers]          # all layer outputs
    functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

    # Testing
    layer_outs = [func([x, 1.]) for func in functors]

    for ln in layer_names:
        i = getLayerIndexByName(model, ln)
        featMatrices.append(layer_outs[i])
        
        
    return featMatrices

def get_content_loss(F, P):
    cLoss = 0.5*K.sum(K.square(F - P))
    return cLoss

In [ ]:
def get_Gram_matrix(F):
    G = K.dot(F, K.transpose(F))
    return G

def get_style_loss(ws, Gs, As):
    sLoss = K.variable(0.)
    for w, G, A in zip(ws, Gs, As):
        M_l = K.int_shape(G)[1]
        N_l = K.int_shape(G)[0]
        G_gram = get_Gram_matrix(G)
        A_gram = get_Gram_matrix(A)
        sLoss+= w*0.25*K.sum(K.square(G_gram - A_gram))/ (N_l**2 * M_l**2)
    return sLoss

In [ ]:
def get_total_loss(gImPlaceholder, alpha=1.0, beta=10000.0):
    F = get_feature_reps(gImPlaceholder, layer_names=[cLayerName], model=gModel)[0]
    Gs = get_feature_reps(gImPlaceholder, layer_names=sLayerNames, model=gModel)
    contentLoss = get_content_loss(F, P)
    styleLoss = get_style_loss(ws, Gs, As)
    totalLoss = alpha*contentLoss + beta*styleLoss
    return totalLoss

In [ ]:
def calculate_loss(gImArr):
    """
    Calculate total loss using K.function
    """
    if gImArr.shape != (1, TARGET_WIDTH, TARGET_WIDTH, 3):
        gImArr = gImArr.reshape((1, TARGET_WIDTH, TARGET_HEIGHT, 3))
    loss_fcn = K.function([gModel.input], [get_total_loss(gModel.input)])
    return loss_fcn([gImArr])[0].astype('float64')

def get_grad(gImArr):
    """
    Calculate the gradient of the loss function with respect to the generated image
    """
    if gImArr.shape != (1, TARGET_WIDTH, TARGET_WIDTH, 3):
        gImArr = gImArr.reshape((1, TARGET_WIDTH, TARGET_HEIGHT, 3))
    grad_fcn = K.function([gModel.input], 
                          K.gradients(get_total_loss(gModel.input), [gModel.input]))
    grad = grad_fcn([gImArr])[0].flatten().astype('float64')
    return grad

# Perform Style Transfer


In [ ]:
model.input

In [ ]:
tf_session = K.get_session()
cModel = get_model()
sModel = get_model()
gModel = get_model()

In [ ]:
cLayerName = 'conv2d_2'
sLayerNames = [
    'conv2d_1',
    'conv2d_2',
]

In [ ]:
sModel.summary()

In [ ]:
sModel.summary()

In [ ]:
P = get_feature_reps(x=cImArr, layer_names=[cLayerName], model=cModel)[0]
As = get_feature_reps(x=sImArr, layer_names=sLayerNames, model=sModel)
ws = np.ones(len(sLayerNames)) / float(len(sLayerNames))

In [ ]:
iterations = 100
x_val = generatedImage.flatten()
xopt, f_val, info= fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad,
                            maxiter=iterations, disp=True)

In [ ]:
xopt.shape

In [ ]:
xopt = xopt.reshape(128, 128, 3)

In [ ]:
from PIL import Image

In [ ]:
img = Image.fromarray(xopt, 'RGB')
img.show()

In [ ]:
# Trial 2

In [ ]:
from keras.applications import VGG16
from scipy.optimize import fmin_l_bfgs_b

In [ ]:
tf_session = K.get_session()

In [ ]:
cModel = VGG16(include_top=False, weights='imagenet', input_tensor=cImArr)
sModel = VGG16(include_top=False, weights='imagenet', input_tensor=sImArr)
gModel = VGG16(include_top=False, weights='imagenet', input_tensor=generatedImagePlaceholder)

In [ ]:
cModel.summary()

In [ ]:
cLayerName = 'block4_conv2'
sLayerNames = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                ]

In [ ]:
P = get_feature_reps(x=cImArr, layer_names=[cLayerName], model=cModel)[0]
As = get_feature_reps(x=sImArr, layer_names=sLayerNames, model=sModel)
ws = np.ones(len(sLayerNames))/float(len(sLayerNames))


In [ ]:
iterations = 4
x_val = generatedImage.flatten()
xopt, f_val, info= fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad,
                            maxiter=iterations, disp=True)

In [ ]:
info

In [ ]:
xopt.shape

In [ ]:
xopt = xopt.reshape(128, 128, 3)

In [ ]:
from PIL import Image

In [ ]:
img = Image.fromarray(xopt, 'RGB')
img.show()